In [1]:
file = open(r"q03.cpp",'r')
#return_type = ("void","int","float","double","long long int","vector","string","stack")
updated_lines = []
change_words={}
for line in file:
    if(not line.startswith("//") and line!="\n"  and not line.startswith('#') and not line.startswith('using')):
        updated_lines.append(line.lower().strip())
    if(line.startswith("#define")):
        words = line.split(' ')
        #print(words)
        replace = words[1]
        string = ""
        for i in range(2,len(words)):
            string += words[i]
            string += " "
        string = string[:-2]
        #print(string)
        change_words[replace] = string

In [2]:
updated_lines

['inline nnt shift_done(stack<int> a, stack<int> b, stack<string> c) {',
 'int a_size = a.size();',
 'int b_size = b.size();',
 'int c_size = c.size();',
 'if(a_size==0){',
 'if(b_size==0 || c_size==0){',
 'return 1;',
 '}',
 'else{',
 'return 0;',
 '}',
 '}',
 'else if(b_size==0 && c_size==0){',
 'return 1;',
 '}',
 'else{',
 'return 0;',
 '}',
 '}',
 'void hanoi_iterative(int n){',
 'stack<int> a;',
 'stack<int> b;',
 'stack<int> c;',
 'for (int i = 0; i < n; i++) {',
 'a.push(n - i);',
 '}',
 'int number = 1;',
 'if (n % 2) {',
 'cout << "a b 1"<<\'\\n\';',
 'b.push(a.top());',
 'a.pop();',
 '}',
 'else {',
 'cout << "a c 1"<<\'\\n\';',
 'c.push(a.top());',
 'a.pop();',
 'number = 2;',
 '}',
 'while(!shift_done(a,b,c)){',
 'int a_top=0, b_top=0, c_top=0;',
 'int a_size=a.size(), b_size=b.size(), c_size=c.size();',
 'if(a_size!=0){',
 'a_top=a.top();',
 '}',
 'if(b_size!=0){',
 'b_top=b.top();',
 '}',
 'if(c_size!=0){',
 'c_top=c.top();',
 '}',
 'if(number%3==0){',
 'if (b_size==0){'

In [10]:
#return_type = {"void ","int ","float ","double ","long long int ","vector ","string ","stack "}
code=""
for i in range(len(updated_lines)):
    code += updated_lines[i]

In [11]:
code

'inline nnt shift_done(stack<int> a, stack<int> b, stack<string> c) {int a_size = a.size();int b_size = b.size();int c_size = c.size();if(a_size==0){if(b_size==0 || c_size==0){return 1;}else{return 0;}}else if(b_size==0 && c_size==0){return 1;}else{return 0;}}void hanoi_iterative(int n){stack<int> a;stack<int> b;stack<int> c;for (int i = 0; i < n; i++) {a.push(n - i);}int number = 1;if (n % 2) {cout << "a b 1"<<\'\\n\';b.push(a.top());a.pop();}else {cout << "a c 1"<<\'\\n\';c.push(a.top());a.pop();number = 2;}while(!shift_done(a,b,c)){int a_top=0, b_top=0, c_top=0;int a_size=a.size(), b_size=b.size(), c_size=c.size();if(a_size!=0){a_top=a.top();}if(b_size!=0){b_top=b.top();}if(c_size!=0){c_top=c.top();}if(number%3==0){if (b_size==0){b.push(c_top);c.pop();cout << "c b " << c_top << \'\\n\';}else if(c_size==0 || (b_top < c_top)){c.push(b_top);b.pop();cout << "b c " << b_top << \'\\n\';}else{b.push(c_top);c.pop();cout << "c b " << c_top << \'\\n\';}if (n%2){cout << "a b 1"<<\'\\n\';b.push

In [12]:
change_words

{'nnt': 'long long int'}

In [13]:
code = code.translate({ord("}"):"}; ",ord("("):" ( ",ord("{"):" { ;"})
for key,value in change_words.items():
    code=code.replace(key,value)

In [14]:
lines_code=code.split(';')

In [15]:
lines_code

['inline long long int shift_done ( stack<int> a, stack<int> b, stack<string> c)  { ',
 'int a_size = a.size ( )',
 'int b_size = b.size ( )',
 'int c_size = c.size ( )',
 'if ( a_size==0) { ',
 'if ( b_size==0 || c_size==0) { ',
 'return 1',
 '}',
 ' else { ',
 'return 0',
 '}',
 ' }',
 ' else if ( b_size==0 && c_size==0) { ',
 'return 1',
 '}',
 ' else { ',
 'return 0',
 '}',
 ' }',
 ' void hanoi_iterative ( int n) { ',
 'stack<int> a',
 'stack<int> b',
 'stack<int> c',
 'for  ( int i = 0',
 ' i < n',
 ' i++)  { ',
 'a.push ( n - i)',
 '}',
 ' int number = 1',
 'if  ( n % 2)  { ',
 'cout << "a b 1"<<\'\\n\'',
 'b.push ( a.top ( ))',
 'a.pop ( )',
 '}',
 ' else  { ',
 'cout << "a c 1"<<\'\\n\'',
 'c.push ( a.top ( ))',
 'a.pop ( )',
 'number = 2',
 '}',
 ' while ( !shift_done ( a,b,c)) { ',
 'int a_top=0, b_top=0, c_top=0',
 'int a_size=a.size ( ), b_size=b.size ( ), c_size=c.size ( )',
 'if ( a_size!=0) { ',
 'a_top=a.top ( )',
 '}',
 ' if ( b_size!=0) { ',
 'b_top=b.top ( )',
 '}',


In [18]:
import re
from collections import Counter
pattern ="\((.*?)\)"
for i in range(len(lines_code)): 
    if ('{'in lines_code[i] and '(' in lines_code[i] and ')' in lines_code[i] and not('for ' in lines_code[i]) 
        and not('while ' in lines_code[i]) and not('main' in lines_code[i]) and not('if ' in lines_code[i])):
        
        substring = re.search(pattern, lines_code[i]).group(1)
        
        count_parameter=[]
        type_parameter=[]
        parameters = (substring.split(','))
        parameter_type = [i.rsplit(' ', 1)[0].strip() for i in parameters]
        count_parameter_all = dict(Counter(parameter_type))
        for key,value in count_parameter_all.items():
            count_parameter.append(value)
            type_parameter.append(key)
        
        
        words = lines_code[i].split(" ")
        if(words[0]=='static' or words[0]=='inline'):
            words[0]=''
        return_type=""
        for j in range(len(words)):
            if(words[j]=='('):
                for k in range(j-1):
                    return_type +=words[k]
                    return_type +=" "
                break

        function_signature=(return_type.strip(),count_parameter,type_parameter)
        print(function_signature)

('long long int', [2, 1], ['stack<int>', 'stack<string>'])
('void', [1], ['int'])
